# 1. Demande
Copie du mail de **** du 17/11/2021 :
>_Lors de la réunion que nous avons eue hier, j’ai évoqué l’enquête CLEF pour disposer de données récentes sur la situation. Pourrais-tu m’envoyer les résultats pour ce qui concerne la connectivité ? (...)J’ai besoin des données pour l’enseignement obligatoire uniquement, ordinaire et spécialisé, fondamental et secondaire._
# 2. Livrables
Un tableau de bord (fréquences simples et brutes) des résultats aux questions de l'enquête qui portent sur la connectivité. Ces résultats doivent concernés uniquement l'enseignement obligatoire et ventilés par type d'enseignement.
# 3. Fichiers source
Les mêmes que dans CLEF-WB - TB PO.ipynb

In [116]:
###################
### Mise en page ##
###################
def formatage_titre(feuille):
    return feuille.add_format({
        'font_size':16,
        'font_color':'blue',
        'align':'center',
        'valign':'vcenter',
        'border':2        
    })

def formatage_question(feuille):
    return feuille.add_format({
    'bold':True,
    'bg_color':'navy',
    'font_color':'white',
    'text_wrap': True,
    'valign': 'top'
    })

def formatage_entete_col(feuille):
    return feuille.add_format({
            'bold':True,
        'align':'center'
    })

def formatage_reponse(feuille):
    return feuille.add_format({
        'align':'right'
    })

def formatage_stats(feuille):
    return feuille.add_format({
        'align':'left',
        'bold':True,
    })
###################
### Traitements ###
###################
def taux_de_reponse(serie_total, obs_total):
    return round((serie_total/obs_total)*100, 2)

def traitement_ON(serie, obs, df, label, bins):
    '''Réponses Ouvertes Numériques'''
    r1 = serie.count()
    t1 = taux_de_reponse(r1, n_obs_i)
    moyenne = round(serie.mean(), 2)
    ecart_type = round(serie.std(), 2)
    min_value = serie.min()
    max_value = serie.max()
    interval = pd.cut(serie, bins=bins, labels=label, duplicates='drop')
    fx = pd.DataFrame(df.groupby(interval).size()).rename(columns={0:'Nb'})
    fx['%obs'] = fx.Nb.apply(lambda x:round((x/r1)*100,2))
    fx.Nb = fx.Nb.astype(int)    
    return fx, t1, moyenne, ecart_type

def traitement_FU(serie, obs, nom_col, df):
    '''Réponses Fermées Uniques'''
    r1 = serie.count()
    tx1 = taux_de_reponse(r1, obs)
    f1 = pd.DataFrame(df.groupby(nom_col).size()).rename(columns={0:'Nb'})
    f1['%obs'] = f1.Nb.apply(lambda x:round((x/r1)*100,2))
    f1.loc['Total'] = round(f1.sum(),0)
    f1.Nb = f1.Nb.astype(int)
    return f1, tx1

def traitement_FM(serie, obs, df):
    '''Réponses Fermées Multiples'''
    r1 = serie.count()
    tx1 = taux_de_reponse(r1, obs)
    f3 = pd.DataFrame(serie.astype(str).str.split(pat=";", expand=True)
                      .stack().value_counts()).rename(columns={0:'Nb'})
    f3 = f3[f3.index != 'nan']
    f3['%obs'] = f3.Nb.apply(lambda x:round((x/r1)*100,2))
    f3.Nb = f3.Nb.astype(int)
    return f3, tx1

def traitement_FE(sqst,col, qst, feuille, frmt1, frmt2, frmt3, df, n_obs):
    '''Réponses Fermées à Echelle'''
    feuille.write(feuille.dim_rowmax + 1,0,qst,frmt1)
    for l in sqst:
        r4 = df[col+l[0]].count()
        tr4 = round((r4/n_obs)*100, 2)
        tx4 = taux_de_reponse(r4, n_obs)
        f4 = pd.DataFrame(df.groupby(col+l[0]).size()).rename(columns={0:'Nb'})
        f4['%obs'] = f4.Nb.apply(lambda x:round((x/r4)*100,2))
        f4.loc['Total'] = round(f4.sum(),0)
        f4.Nb = f4.Nb.astype(int)
        inscription_question(feuille, frmt1, frmt2, tx4, col+l[0]+' : '+l[1])
        inscription_resultats(f4, feuille.dim_rowmax + 1, frmt3, feuille)

################
### Ecriture ###
################  
def inscription_resultats(df,ligne,frmt,sheet):
    for x in df.itertuples():
        sheet.write(ligne,col,x[0],frmt)
        sheet.write(ligne,col+1,x[1])
        sheet.write(ligne,col+2,x[2])
        ligne += 1

def inscription_question(sheet, frmt1, frmt2, tdr, question):
    dbt_nv_tbl = sheet.dim_rowmax + 1
    sheet.write(dbt_nv_tbl,0,question,frmt1)
    sheet.write(dbt_nv_tbl+1,1,'Nb',frmt2)
    sheet.write(dbt_nv_tbl+1,2,'%obs',frmt2)
    sheet.write(dbt_nv_tbl+1,0,'Taux de réponse: '+str(tdr)+'%')

In [42]:
import os
import sys
import utils
import pandas as pd
import xlsxwriter
from xlsxwriter import Workbook
root = r"C:\dev2022\fwb\data\CellArchi\sonecom\données brutes"
po_file = r"\Export_BD complète_PO.xlsx"
bat_file_old = r"\Export_BD complète_Batiment.xlsx" # contient informations
bat_file = r"\CLEF-WB - Export et TB - BD Batiment vérifiée.xlsx" # chiffres corrigées
impl_file = r"\Export_BD complète _Implantation.xlsx"

In [43]:
# snippet pour permettre l'importation des fonctions logées dans utils.py
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
p_df = pd.read_excel(root+po_file)
b_df = pd.read_excel(root+bat_file)
i_df = pd.read_excel(root+impl_file)
dfs = (p_df, b_df, i_df)

# 4. Sélection des questions sur la connectivité
Les questions relatives à l'Axe Numérique de l'enquête sont présentes dans les questionnaires PO et Implantations. Dans le questionnaire Bâtiment, pas de questions ouvertement sur le sujet. (à moins de considérer l'item _"Learning Center (ou assimilés)"_ des questions 12, 13 et 14 comme portant sur la connectivité)

In [129]:
import questions
# Questions PO
q6 = questions.q6
# Questions IMPLANTATION
q29 = questions.q29
q30 = questions.q30
q31 = questions.q31
q32 = questions.q32

# 5. Sélection des participants concernés

### 5.1. Pour le questionnaire PO
On utilise la variable **"niveau"**, qui comporte 5 modalités : 
- 'Fondamental', 'Secondaire', 'Supérieur', 'Autre'

et la variable **"nature"** et 3 modalités :
- 'Etablissement scolaire','C.P.M.S./Service P.S.E.','Internat'

Dans ce cas, on utilise ces deux variables pour filtrer les données

### 5.2. Pour le questionnaire IMPLANTATIONS
Plusieurs variables contiennent l'information utile, déclinées en différentes modalités :
- **nature** :
    - 'Etablissement scolaire','C.P.M.S./Service P.S.E.','Internat'
- **niveau** : 
    - 'Fondamental', 'Secondaire', 'Supérieur', 'Autre'
- **nom_unite_imp_ecole** : 
    - 'Primaire ordinaire',
    - 'Secondaire ordinaire',
    - 'Artistique à horaire réduit',
    - 'Secondaire spécialisé',
    - 'Maternel ordinaire',
    - 'Primaire spécialisé',
    - 'Internat secondaire',
    - 'Secondaire CEFA',
    - 'Promotion sociale secondaire',
    - 'École supérieure des Arts',
    - 'Haute École',
    - 'Home permanent',
    - 'Promotion sociale supérieur',
    - 'Maternel spécialisé',
    - 'Centre PMS',
    - 'Internat spécialisé',
    - 'Internat supérieur',
    - 'Internat primaire',
    - 'ISA',
    - 'Promotion Sociale CEFA'
- **categorie_imp** :
    - 'Plein exercice', 'Horaire réduit', 'Internat', 'Promotion Sociale', 'Centre PMS'
- **genre_imp** :
    - 'Ordinaire', 'Spécialisé', 'Autre'
- **niveau_imp** :
    - 'Fondamental', 'Secondaire', 'Supérieur', 'Autre'
    
**nature** a les mêmes modalités que **nature** dans questionnaire PO. **niveau_imp** et **niveau** sont les mêmes variables. On va donc utiliser les variables **nature**, **niveau** et **genre_imp**. La variable **nom_unite_imp_ecole** pourra être utilisée pour des résultats plus fins.

In [18]:
# 1. Questionnaire PO
# Nombre de PO
n_p = len(p_df)
# Nombre de répondants 578
n_p_p = p_df[p_df.Participation == "PO participant"].shape[0]
# Taux de répondants 53,37%
tx_p = round((n_p_p/n_p)*100, 2)
# 2. Questionnaire Implantations
# Nombre d'implantations 8719
n_i = len(i_df)
# Nombre de répondants 5425
n_p_i = i_df[i_df.Participation == "Participant"].shape[0]
# Taux de répondants 62,22%%
tx_i = round((n_p_i/n_i)*100, 2)

In [19]:
p_sco = p_df[(p_df.Participation == "PO participant")
             & (p_df.nature == "Etablissement scolaire")
             & (p_df.niveau != "Supérieur" )]
i_sco = i_df[(i_df.Participation == "Participant") 
            & (i_df.nature == "Etablissement scolaire") 
            & (i_df.niveau != "Supérieur")]

# 6. Traitements
**N.B.** : Une mise en page manuelle sera appliquée sur le fichier produit.

In [128]:
cahier = pd.ExcelWriter(r"C:\dev2022\fwb\jupyter\CelluleArchi\axe_numerique.xlsx",
            engine="xlsxwriter",
            engine_kwargs={'options':{'nan_inf_to_errors': True}})
wb = cahier.book
feuille = wb.add_worksheet('Axe Numérique')
# Mise en page
frmt_titre = formatage_titre(wb)
frmt_quest = formatage_question(wb)
frmt_entete_colonne = formatage_entete_col(wb)
frmt_rep = formatage_reponse(wb)
frmt_stat = formatage_stats(wb)
# Titre
feuille.set_column('A1:A1',55)
feuille.merge_range('A1:C1',"Axe Numérique - établissements scolaires",frmt_titre)
feuille.merge_range('A2:C2',"Nombre de PO participants : "
                    +str(n_p_p)+" ("+str(tx_p)+"% des "+str(n_p)
                    +" PO recensés)")
feuille.merge_range("A3:C3","Nombre d'Implantations participantes : "
                   +str(n_p_i)+" ("+str(tx_i)+"% des "+str(n_i)
                   +" Implantations recensées)")
feuille.merge_range("A4:C4","Nombre de PO Scolaires (hors supérieur) : "
                   +str(len(p_sco)))
feuille.merge_range("A5:C5","Nombre d'Implantations scolaires (hors supérieur) : "
                   +str(len(i_sco)))
feuille.set_row(0,25)
col = 0
# 1ère question
# "6. Dans le cadre du PEE, avez-vous la volonté de faire 
# évoluer vos bâtiments au regard du numérique ?"
feuille.write(feuille.dim_rowmax + 2,col,q6,frmt_quest)
for niveau in p_sco.niveau.unique():
    df = p_sco[p_sco.niveau == niveau]
    serie = df.Q06_b
    tx6 = taux_de_reponse(serie.count(),len(df)) 
    f6 = pd.DataFrame(df.groupby(serie).size()).rename(columns={0:'Nb'})
    f6['%obs'] = f6.Nb.apply(lambda x:round((x/serie.count())*100,2))
    f6.loc['Total'] = round(f6.sum(),0)
    f6.Nb = f6.Nb.astype(int)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx6, niveau)
    inscription_resultats(f6, feuille.dim_rowmax + 1, frmt_rep, feuille)
# 2ème question
feuille.write(feuille.dim_rowmax + 2,col,q29,frmt_quest)
# Réponses ventilées par niveau
for niveau in i_sco.niveau.unique():
    df_n = i_sco[i_sco.niveau == niveau]
    f29_n, tx29_n = traitement_FM(df_n.Q19_cat, len(df_n), df_n)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx29_n, niveau)
    inscription_resultats(f29_n, feuille.dim_rowmax + 1, frmt_rep, feuille)
# Réponses ventilées par genre
for genre in i_sco.genre_imp.unique():
    df_g = i_sco[i_sco.genre_imp == genre]
    f29_g, tx29_g = traitement_FM(df_g.Q19_cat, len(df_g), df_g)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx29_g, genre)
    inscription_resultats(f29_g, feuille.dim_rowmax + 1, frmt_rep, feuille)
# 3ème question
feuille.write(feuille.dim_rowmax + 2,col,q30,frmt_quest)
# Réponses ventilées par niveau
for niveau in i_sco.niveau.unique():
    df = i_sco[i_sco.niveau == niveau]
    f30, tx30 = traitement_FU(df.Q20, len(df), 'Q20', df)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx30, niveau)
    inscription_resultats(f30, feuille.dim_rowmax + 1, frmt_rep, feuille)
# Réponses ventilées par genre
for genre in i_sco.genre_imp.unique():
    df_g = i_sco[i_sco.genre_imp == genre]
    f30_g, tx30_g = traitement_FU(df_g.Q20, len(df_g), 'Q20', df_g)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx30_g, genre)
    inscription_resultats(f30_g, feuille.dim_rowmax + 1, frmt_rep, feuille)
# 4ème question
feuille.write(feuille.dim_rowmax + 2,col,q31,frmt_quest)
# Réponses ventilées par niveau
for niveau in i_sco.niveau.unique():
    df = i_sco[i_sco.niveau == niveau]
    f31, tx31 = traitement_FU(df.Q21, len(df), 'Q21', df)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx31, niveau)
    inscription_resultats(f31, feuille.dim_rowmax + 1, frmt_rep, feuille)
# Réponses ventilées par genre
for genre in i_sco.genre_imp.unique():
    df_g = i_sco[i_sco.genre_imp == genre]
    f31_g, tx31_g = traitement_FU(df_g.Q21, len(df_g), 'Q21', df_g)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx31, genre)
    inscription_resultats(f31, feuille.dim_rowmax + 1, frmt_rep, feuille)
#5ème question
feuille.write(feuille.dim_rowmax + 2,col,q32,frmt_quest)
# Réponses ventilées par niveau
for niveau in i_sco.niveau.unique():
    df = i_sco[i_sco.niveau == niveau]
    f32, tx32 = traitement_FU(df.Q22, len(df), 'Q22', df)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx32, niveau)
    inscription_resultats(f32, feuille.dim_rowmax + 1, frmt_rep, feuille)
# Réponses ventilées par genre
for genre in i_sco.genre_imp.unique():
    df_g = i_sco[i_sco.genre_imp == genre]
    f32_g, tx32_g = traitement_FU(df_g.Q22, len(df_g), 'Q22', df_g)
    inscription_question(feuille, frmt_quest, frmt_entete_colonne, tx32, genre)
    inscription_resultats(f32, feuille.dim_rowmax + 1, frmt_rep, feuille)
# Fin des traitements
cahier.save()